# Working Game Results

This notebook contains:
- Importing results from historical data
- Data manipulation in preparation for merging
- Additional steps in EDA

This notebook is essential for determining the accuracy of our predictions and giving us a target 

## Table of Contents
**1. Importing and cleaning game results**
- Removing games for teams that no longer exist

**2. Exporting in a format that can be easily merged on other datasets**

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sportsreference as csr
import html5lib

import requests
from bs4 import BeautifulSoup

In [56]:
games = pd.read_csv('/Users/justintunley/Documents/BrainStation/Capstone/Data Sets/games.csv')

In [57]:
games.head()

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,ftn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
0,1999_01_MIN_ATL,1999,REG,1,1999-09-12,Sunday,NaN,MIN,17.0,ATL,14.0,Home,-3.0,31.0,0.0,1999091210,598.0,NaN,199909120atl,NaN,190912001,NaN,7,7,NaN,NaN,-4.0,NaN,NaN,49.0,NaN,NaN,0,dome,astroturf,NaN,NaN,00-0003761,00-0002876,Randall Cunningham,Chris Chandler,Dennis Green,Dan Reeves,Gerry Austin,ATL00,Georgia Dome
1,1999_01_KC_CHI,1999,REG,1,1999-09-12,Sunday,NaN,KC,17.0,CHI,20.0,Home,3.0,37.0,0.0,1999091206,597.0,NaN,199909120chi,NaN,190912003,NaN,7,7,NaN,NaN,-3.0,NaN,NaN,38.0,NaN,NaN,0,outdoors,grass,80.0,12.0,00-0006300,00-0010560,Elvis Grbac,Shane Matthews,Gunther Cunningham,Dick Jauron,Phil Luckett,CHI98,Soldier Field
2,1999_01_PIT_CLE,1999,REG,1,1999-09-12,Sunday,NaN,PIT,43.0,CLE,0.0,Home,-43.0,43.0,0.0,1999091213,604.0,NaN,199909120cle,NaN,190912005,NaN,7,7,NaN,NaN,-6.0,NaN,NaN,37.0,NaN,NaN,1,outdoors,grass,78.0,12.0,00-0015700,00-0004230,Kordell Stewart,Ty Detmer,Bill Cowher,Chris Palmer,Bob McElwee,CLE00,Cleveland Browns Stadium
3,1999_01_OAK_GB,1999,REG,1,1999-09-12,Sunday,NaN,OAK,24.0,GB,28.0,Home,4.0,52.0,0.0,1999091208,602.0,NaN,199909120gnb,NaN,190912009,NaN,7,7,NaN,NaN,9.0,NaN,NaN,43.0,NaN,NaN,0,outdoors,grass,67.0,10.0,00-0005741,00-0005106,Rich Gannon,Brett Favre,Jon Gruden,Ray Rhodes,Tony Corrente,GNB00,Lambeau Field
4,1999_01_BUF_IND,1999,REG,1,1999-09-12,Sunday,NaN,BUF,14.0,IND,31.0,Home,17.0,45.0,0.0,1999091202,591.0,NaN,199909120clt,NaN,190912011,NaN,7,7,NaN,NaN,-3.0,NaN,NaN,45.5,NaN,NaN,1,dome,astroturf,NaN,NaN,00-0005363,00-0010346,Doug Flutie,Peyton Manning,Wade Phillips,Jim Mora,Ron Blum,IND99,RCA Dome


In [58]:
games.shape

(6693, 46)

In [59]:
games.isna().sum()
# a lot of this doesn't matter but some of it definitely does....

# Problems:
# 109 missing scores --> can drop

game_id                0
season                 0
game_type              0
week                   0
gameday                0
weekday                0
gametime             259
away_team              0
away_score           109
home_team              0
home_score           109
location               0
result               109
total                109
overtime             109
old_game_id            0
gsis                 569
nfl_detail_id       6421
pfr                    0
pff                 2401
espn                   0
ftn                 5589
away_rest              0
home_rest              0
away_moneyline      2075
home_moneyline      2075
spread_line           94
away_spread_odds    2075
home_spread_odds    2075
total_line            94
under_odds          2078
over_odds           2078
div_game               0
roof                  18
surface               41
temp                1937
wind                1937
away_qb_id           100
home_qb_id           100
away_qb_name         100


In [60]:
pd.set_option('display.max_columns',None)
games.head()

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,ftn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
0,1999_01_MIN_ATL,1999,REG,1,1999-09-12,Sunday,NaN,MIN,17.0,ATL,14.0,Home,-3.0,31.0,0.0,1999091210,598.0,NaN,199909120atl,NaN,190912001,NaN,7,7,NaN,NaN,-4.0,NaN,NaN,49.0,NaN,NaN,0,dome,astroturf,NaN,NaN,00-0003761,00-0002876,Randall Cunningham,Chris Chandler,Dennis Green,Dan Reeves,Gerry Austin,ATL00,Georgia Dome
1,1999_01_KC_CHI,1999,REG,1,1999-09-12,Sunday,NaN,KC,17.0,CHI,20.0,Home,3.0,37.0,0.0,1999091206,597.0,NaN,199909120chi,NaN,190912003,NaN,7,7,NaN,NaN,-3.0,NaN,NaN,38.0,NaN,NaN,0,outdoors,grass,80.0,12.0,00-0006300,00-0010560,Elvis Grbac,Shane Matthews,Gunther Cunningham,Dick Jauron,Phil Luckett,CHI98,Soldier Field
2,1999_01_PIT_CLE,1999,REG,1,1999-09-12,Sunday,NaN,PIT,43.0,CLE,0.0,Home,-43.0,43.0,0.0,1999091213,604.0,NaN,199909120cle,NaN,190912005,NaN,7,7,NaN,NaN,-6.0,NaN,NaN,37.0,NaN,NaN,1,outdoors,grass,78.0,12.0,00-0015700,00-0004230,Kordell Stewart,Ty Detmer,Bill Cowher,Chris Palmer,Bob McElwee,CLE00,Cleveland Browns Stadium
3,1999_01_OAK_GB,1999,REG,1,1999-09-12,Sunday,NaN,OAK,24.0,GB,28.0,Home,4.0,52.0,0.0,1999091208,602.0,NaN,199909120gnb,NaN,190912009,NaN,7,7,NaN,NaN,9.0,NaN,NaN,43.0,NaN,NaN,0,outdoors,grass,67.0,10.0,00-0005741,00-0005106,Rich Gannon,Brett Favre,Jon Gruden,Ray Rhodes,Tony Corrente,GNB00,Lambeau Field
4,1999_01_BUF_IND,1999,REG,1,1999-09-12,Sunday,NaN,BUF,14.0,IND,31.0,Home,17.0,45.0,0.0,1999091202,591.0,NaN,199909120clt,NaN,190912011,NaN,7,7,NaN,NaN,-3.0,NaN,NaN,45.5,NaN,NaN,1,dome,astroturf,NaN,NaN,00-0005363,00-0010346,Doug Flutie,Peyton Manning,Wade Phillips,Jim Mora,Ron Blum,IND99,RCA Dome


In [61]:
testYear = games['game_id'].str.split('_', expand=True)
testYear.head()
# splitting the game_id along the '_'

,0,1,2,3
0,1999,01,MIN,ATL
1,1999,01,KC,CHI
2,1999,01,PIT,CLE
3,1999,01,OAK,GB
4,1999,01,BUF,IND


In [62]:
games['year'] = testYear[0]
# I realized after there was a seasons column so this was kinda unnecessary but oh well

In [63]:
games.tail()

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,ftn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium,year
6688,2023_18_ATL_NO,2023,REG,18,2024-01-07,Sunday,13:00,ATL,NaN,NO,NaN,Home,NaN,NaN,NaN,2022121711,NaN,NaN,202212170nor,NaN,401547649,NaN,13,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,dome,sportturf,NaN,NaN,NaN,NaN,NaN,NaN,Arthur Smith,Dennis Allen,NaN,NOR00,Mercedes-Benz Superdome,2023
6689,2023_18_PHI_NYG,2023,REG,18,2024-01-07,Sunday,13:00,PHI,NaN,NYG,NaN,Home,NaN,NaN,NaN,2022121712,NaN,NaN,202212170nyg,NaN,401547650,NaN,6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,outdoors,fieldturf,NaN,NaN,NaN,NaN,NaN,NaN,Nick Sirianni,Brian Daboll,NaN,NYC01,MetLife Stadium,2023
6690,2023_18_LA_SF,2023,REG,18,2024-01-07,Sunday,13:00,LA,NaN,SF,NaN,Home,NaN,NaN,NaN,2022121713,NaN,NaN,202212170sfo,NaN,401547651,NaN,9,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,outdoors,grass,NaN,NaN,NaN,NaN,NaN,NaN,Sean McVay,Kyle Shanahan,NaN,SFO01,Levi's Stadium,2023
6691,2023_18_JAX_TEN,2023,REG,18,2024-01-07,Sunday,13:00,JAX,NaN,TEN,NaN,Home,NaN,NaN,NaN,2022121714,NaN,NaN,202212170oti,NaN,401547652,NaN,6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,outdoors,grass,NaN,NaN,NaN,NaN,NaN,NaN,Doug Pederson,Mike Vrabel,NaN,NAS00,Nissan Stadium,2023
6692,2023_18_DAL_WAS,2023,REG,18,2024-01-07,Sunday,13:00,DAL,NaN,WAS,NaN,Home,NaN,NaN,NaN,2022121715,NaN,NaN,202212170was,NaN,401547653,NaN,6,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,outdoors,grass,NaN,NaN,NaN,NaN,NaN,NaN,Mike McCarthy,Ron Rivera,NaN,WAS00,FedExField,2023


In [64]:
games.isna().sum()

game_id                0
season                 0
game_type              0
week                   0
gameday                0
weekday                0
gametime             259
away_team              0
away_score           109
home_team              0
home_score           109
location               0
result               109
total                109
overtime             109
old_game_id            0
gsis                 569
nfl_detail_id       6421
pfr                    0
pff                 2401
espn                   0
ftn                 5589
away_rest              0
home_rest              0
away_moneyline      2075
home_moneyline      2075
spread_line           94
away_spread_odds    2075
home_spread_odds    2075
total_line            94
under_odds          2078
over_odds           2078
div_game               0
roof                  18
surface               41
temp                1937
wind                1937
away_qb_id           100
home_qb_id           100
away_qb_name         100


In [65]:
regGames = games.loc[games['game_type'] == 'REG']
# only regular season games

In [66]:
test = regGames[regGames.duplicated(subset=['away_team', 'home_team', 'year'], keep = False)]
# dataframe only containing the duplicated rows in the regular season

In [67]:
test.shape
# no rows so all repeats are in the postseason, which wont matter since our model is regular season only

(0, 47)

In [68]:
# filter out everything beside home team, away team, scores, 
# create 2 columns with home team + score and away team + away score

In [69]:
regGames.head()

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,home_score,location,result,total,overtime,old_game_id,gsis,nfl_detail_id,pfr,pff,espn,ftn,away_rest,home_rest,away_moneyline,home_moneyline,spread_line,away_spread_odds,home_spread_odds,total_line,under_odds,over_odds,div_game,roof,surface,temp,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium,year
0,1999_01_MIN_ATL,1999,REG,1,1999-09-12,Sunday,NaN,MIN,17.0,ATL,14.0,Home,-3.0,31.0,0.0,1999091210,598.0,NaN,199909120atl,NaN,190912001,NaN,7,7,NaN,NaN,-4.0,NaN,NaN,49.0,NaN,NaN,0,dome,astroturf,NaN,NaN,00-0003761,00-0002876,Randall Cunningham,Chris Chandler,Dennis Green,Dan Reeves,Gerry Austin,ATL00,Georgia Dome,1999
1,1999_01_KC_CHI,1999,REG,1,1999-09-12,Sunday,NaN,KC,17.0,CHI,20.0,Home,3.0,37.0,0.0,1999091206,597.0,NaN,199909120chi,NaN,190912003,NaN,7,7,NaN,NaN,-3.0,NaN,NaN,38.0,NaN,NaN,0,outdoors,grass,80.0,12.0,00-0006300,00-0010560,Elvis Grbac,Shane Matthews,Gunther Cunningham,Dick Jauron,Phil Luckett,CHI98,Soldier Field,1999
2,1999_01_PIT_CLE,1999,REG,1,1999-09-12,Sunday,NaN,PIT,43.0,CLE,0.0,Home,-43.0,43.0,0.0,1999091213,604.0,NaN,199909120cle,NaN,190912005,NaN,7,7,NaN,NaN,-6.0,NaN,NaN,37.0,NaN,NaN,1,outdoors,grass,78.0,12.0,00-0015700,00-0004230,Kordell Stewart,Ty Detmer,Bill Cowher,Chris Palmer,Bob McElwee,CLE00,Cleveland Browns Stadium,1999
3,1999_01_OAK_GB,1999,REG,1,1999-09-12,Sunday,NaN,OAK,24.0,GB,28.0,Home,4.0,52.0,0.0,1999091208,602.0,NaN,199909120gnb,NaN,190912009,NaN,7,7,NaN,NaN,9.0,NaN,NaN,43.0,NaN,NaN,0,outdoors,grass,67.0,10.0,00-0005741,00-0005106,Rich Gannon,Brett Favre,Jon Gruden,Ray Rhodes,Tony Corrente,GNB00,Lambeau Field,1999
4,1999_01_BUF_IND,1999,REG,1,1999-09-12,Sunday,NaN,BUF,14.0,IND,31.0,Home,17.0,45.0,0.0,1999091202,591.0,NaN,199909120clt,NaN,190912011,NaN,7,7,NaN,NaN,-3.0,NaN,NaN,45.5,NaN,NaN,1,dome,astroturf,NaN,NaN,00-0005363,00-0010346,Doug Flutie,Peyton Manning,Wade Phillips,Jim Mora,Ron Blum,IND99,RCA Dome,1999


In [70]:
regGames['year'].unique()

array(['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023'], dtype=object)

In [71]:
regGames['season'].unique()

array([1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023])

In [72]:
checkpoint = regGames.drop(['week', 'weekday', 'gametime', 'game_type', 'location', 'overtime', 'old_game_id', 'gsis', 'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_rest', 'home_rest', 'away_moneyline', 'home_moneyline', 'spread_line', 'away_spread_odds', 'home_spread_odds', 'under_odds', 'over_odds', 'roof', 'surface', 'temp', 'wind', 'away_qb_id', 'home_qb_id', 'away_qb_name', 'home_qb_name', 'away_coach','home_coach', 'referee', 'stadium_id', 'stadium'], axis=1)

In [73]:
checkpoint.head()

,game_id,season,gameday,away_team,away_score,home_team,home_score,result,total,total_line,div_game,year
0,1999_01_MIN_ATL,1999,1999-09-12,MIN,17.0,ATL,14.0,-3.0,31.0,49.0,0,1999
1,1999_01_KC_CHI,1999,1999-09-12,KC,17.0,CHI,20.0,3.0,37.0,38.0,0,1999
2,1999_01_PIT_CLE,1999,1999-09-12,PIT,43.0,CLE,0.0,-43.0,43.0,37.0,1,1999
3,1999_01_OAK_GB,1999,1999-09-12,OAK,24.0,GB,28.0,4.0,52.0,43.0,0,1999
4,1999_01_BUF_IND,1999,1999-09-12,BUF,14.0,IND,31.0,17.0,45.0,45.5,1,1999


In [74]:
games = checkpoint.drop(['gameday', 'result', 'total', 'total_line', 'year'], axis=1)

In [75]:
games.head()

,game_id,season,away_team,away_score,home_team,home_score,div_game
0,1999_01_MIN_ATL,1999,MIN,17.0,ATL,14.0,0
1,1999_01_KC_CHI,1999,KC,17.0,CHI,20.0,0
2,1999_01_PIT_CLE,1999,PIT,43.0,CLE,0.0,1
3,1999_01_OAK_GB,1999,OAK,24.0,GB,28.0,0
4,1999_01_BUF_IND,1999,BUF,14.0,IND,31.0,1


In [76]:
# make team abbrevs lowercase for join

In [77]:
games['away_team'] = games['away_team'].str.lower()

In [78]:
games['home_team'] = games['home_team'].str.lower()

In [79]:
games.head()

,game_id,season,away_team,away_score,home_team,home_score,div_game
0,1999_01_MIN_ATL,1999,min,17.0,atl,14.0,0
1,1999_01_KC_CHI,1999,kc,17.0,chi,20.0,0
2,1999_01_PIT_CLE,1999,pit,43.0,cle,0.0,1
3,1999_01_OAK_GB,1999,oak,24.0,gb,28.0,0
4,1999_01_BUF_IND,1999,buf,14.0,ind,31.0,1


In [80]:
# create 2 columns with home team + score and away team + away score
# I can probably drop game_id as well

In [81]:
games['away_score'] = games['away_score'].astype(str)
games['home_score'] = games['home_score'].astype(str)

In [82]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6423 entries, 0 to 6692
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   game_id     6423 non-null   object
 1   season      6423 non-null   int64 
 2   away_team   6423 non-null   object
 3   away_score  6423 non-null   object
 4   home_team   6423 non-null   object
 5   home_score  6423 non-null   object
 6   div_game    6423 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 401.4+ KB


In [83]:
# games['away_sum'] = ['{}_{}'.format('away_team','away_score')]
# this is another method for the next cell, but unnecessary

In [84]:
games['away_summary'] = games['away_team'] + '_' + games['away_score']

In [85]:
games['home_summary'] = games['home_team'] + '_' + games['home_score']

In [86]:
games.head()

,game_id,season,away_team,away_score,home_team,home_score,div_game,away_summary,home_summary
0,1999_01_MIN_ATL,1999,min,17.0,atl,14.0,0,min_17.0,atl_14.0
1,1999_01_KC_CHI,1999,kc,17.0,chi,20.0,0,kc_17.0,chi_20.0
2,1999_01_PIT_CLE,1999,pit,43.0,cle,0.0,1,pit_43.0,cle_0.0
3,1999_01_OAK_GB,1999,oak,24.0,gb,28.0,0,oak_24.0,gb_28.0
4,1999_01_BUF_IND,1999,buf,14.0,ind,31.0,1,buf_14.0,ind_31.0


In [87]:
games = games.drop(['game_id'], axis=1)

In [88]:
games

,season,away_team,away_score,home_team,home_score,div_game,away_summary,home_summary
0,1999,min,17.0,atl,14.0,0,min_17.0,atl_14.0
1,1999,kc,17.0,chi,20.0,0,kc_17.0,chi_20.0
2,1999,pit,43.0,cle,0.0,1,pit_43.0,cle_0.0
3,1999,oak,24.0,gb,28.0,0,oak_24.0,gb_28.0
4,1999,buf,14.0,ind,31.0,1,buf_14.0,ind_31.0
...,...,...,...,...,...,...,...,...
6688,2023,atl,nan,no,nan,1,atl_nan,no_nan
6689,2023,phi,nan,nyg,nan,1,phi_nan,nyg_nan
6690,2023,la,nan,sf,nan,1,la_nan,sf_nan
6691,2023,jax,nan,ten,nan,1,jax_nan,ten_nan


In [89]:
games.isna().sum()

season          0
away_team       0
away_score      0
home_team       0
home_score      0
div_game        0
away_summary    0
home_summary    0
dtype: int64

In [90]:
games['home_score'].value_counts()


20.0    460
24.0    421
17.0    392
27.0    381
31.0    325
       ... 
53.0      2
5.0       2
62.0      1
2.0       1
70.0      1
Name: home_score, Length: 61, dtype: int64

In [91]:
games.loc[games['home_score']=='nan'].shape

(109, 8)

In [92]:
games = games.loc[games['home_score'] !='nan']

In [93]:
games.head()

,season,away_team,away_score,home_team,home_score,div_game,away_summary,home_summary
0,1999,min,17.0,atl,14.0,0,min_17.0,atl_14.0
1,1999,kc,17.0,chi,20.0,0,kc_17.0,chi_20.0
2,1999,pit,43.0,cle,0.0,1,pit_43.0,cle_0.0
3,1999,oak,24.0,gb,28.0,0,oak_24.0,gb_28.0
4,1999,buf,14.0,ind,31.0,1,buf_14.0,ind_31.0


In [95]:
games.to_csv('all_games.csv', index=False)